![title](DEP_LOGO_2.jpg)

# Trend Station Water Quality Data 
## Web Application Demo

In [1]:
#HIDDEN

import ipywidgets as widgets
from ipywidgets import interact
import pandas as pd
import csv
from geopy.geocoders import Nominatim
from time import sleep
import folium
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from folium.plugins import MarkerCluster
import seaborn as sns
from scipy import stats

In [11]:
#HIDDEN

def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    return unique

df_comb = pd.read_csv('TS_New_Combined_Final.csv')
df_comb = df_comb.fillna(0)

output_three = widgets.Output()
output_four = widgets.Output()

dropdown_stations = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df_comb.station_id), description='Trend Station')

def common_filtering1(station):
    
    output_three.clear_output()
    output_four.clear_output()
    
    df_comb = pd.read_csv('TS_New_Combined_Final.csv')
    df_comb = df_comb.fillna(0)
    
    common_filter1 = df_comb[(df_comb.station_id == station)]
    
    ts = dropdown_stations.value
    
    df_comb = df_comb[df_comb['station_id'].str.contains(str(ts), na=False)]
        
    group = df_comb.groupby('param_name')
    df2 = pd.DataFrame(group.apply(lambda x: x['test_batch_id'].unique()))

    df2.reset_index(inplace=True)
    df2 = df2.rename({0: 'DATES'}, axis='columns')

    dates = df2[df2.param_name == 'BERYLLIUM'].DATES.item()

    initial_date = dates[0]
    
    with output_three:
        display(common_filter1)
        
    with output_four:
        display(dates)

def dropdown_stations_eventhandler(change):
    common_filtering1(change.new)

dropdown_stations.observe(dropdown_stations_eventhandler, names='value')  

display(dropdown_stations)

Dropdown(description='Trend Station', options=('TS001', 'TS002', 'TS003', 'TS004', 'TS005', 'TS006', 'TS007', …

In [12]:
#HIDDEN

display(output_three)

Output()

In [13]:
#HIDDEN

display(output_four)

Output()

In [104]:
#HIDDEN

ts = dropdown_stations.value

output_four = widgets.Output()

df_comb = df_comb[df_comb['station_id'].str.contains(str(ts), na=False)]
        
group = df_comb.groupby('param_name')
df2 = pd.DataFrame(group.apply(lambda x: x['test_batch_id'].unique()))

df2.reset_index(inplace=True)
df2 = df2.rename({0: 'DATES'}, axis='columns')

dates = df2[df2.param_name == 'BERYLLIUM'].DATES.item()

initial_date = dates[0]

df_comb_date = common_filter1
df_comb_date = df_comb_date[df_comb_date['test_batch_id'] == initial_date]
df_comb_date = df_comb_date.drop(['station_id', 'sample_id', 'fraction', 'result_unit', 'test_batch_id'], axis=1)
df_comb_date = df_comb_date.set_index('param_name').transpose()

#df_comb_date = df_comb_date.astype('float')
df_comb_date.columns = ['FIELD ph', 'FIELD CONDUCTIVITY','TEMPERATURE','FLOW', 'LAB CONDUCTIVITY', 'SULFATE', 'DISSOLVED SOLIDS', 'SUSPENDED SOLIDS', 'LAB ph', 'ALKALINITY', 'HOT ACIDITY', 'MINERAL ACIDITY', 'IRON_D', 'MANGANESE_D', 'ALUMINUM_D', 'IRON_T', 'MANGANESE_T', 'ALUMINUM_T', 'ANTIMONY', 'ARSENIC', 'BERYLLIUM', 'CADMIUM', 'CHROMIUM', 'COPPER', 'LEAD', 'MERCURY', 'NICKEL', 'SELENIUM', 'SILVER', 'THALLIUM', 'ZINC', 'CYANIDE', 'PHENOLS', 'NITRATE', 'SODIUM', 'CALCIUM', 'MAGNESIUM', 'POTASSIUM', 'BI-CARBONATE', 'CARBONATE', 'PHOSPHATE']

df_comb_concat = []
df_comb_concat = pd.concat([df_comb_date])

as_list = df_comb_concat.index.tolist()
idx = as_list.index('result_value')
as_list[idx] = initial_date
df_comb_concat.index = as_list

for i, date in enumerate(dates):
    
    df_comb_date = common_filter1
    df_comb_date = df_comb_date[df_comb_date['test_batch_id'] == dates[i]]
    df_comb_date = df_comb_date.drop(['station_id', 'sample_id', 'fraction', 'result_unit', 'test_batch_id'], axis=1)
    df_comb_date = df_comb_date.set_index('param_name').transpose()
    
    #df_comb_date = df_comb_date.astype('float')

    df_comb_date.columns = ['FIELD ph', 'FIELD CONDUCTIVITY','TEMPERATURE','FLOW', 'LAB CONDUCTIVITY', 'SULFATE', 'DISSOLVED SOLIDS', 'SUSPENDED SOLIDS', 'LAB ph', 'ALKALINITY', 'HOT ACIDITY', 'MINERAL ACIDITY', 'IRON_D', 'MANGANESE_D', 'ALUMINUM_D', 'IRON_T', 'MANGANESE_T', 'ALUMINUM_T', 'ANTIMONY', 'ARSENIC', 'BERYLLIUM', 'CADMIUM', 'CHROMIUM', 'COPPER', 'LEAD', 'MERCURY', 'NICKEL', 'SELENIUM', 'SILVER', 'THALLIUM', 'ZINC', 'CYANIDE', 'PHENOLS', 'NITRATE', 'SODIUM', 'CALCIUM', 'MAGNESIUM', 'POTASSIUM', 'BI-CARBONATE', 'CARBONATE', 'PHOSPHATE']
    
    df_comb_concat = pd.concat([df_comb_concat, df_comb_date])
    
    as_list = df_comb_concat.index.tolist()
    idx = as_list.index('result_value')
    as_list[idx] = dates[i]
    df_comb_concat.index = as_list

    df_comb_concat.index.name = str(dropdown_stations.value)
        
df_comb_concat = df_comb_concat.fillna(0)
    
df_comb_concat = df_comb_concat.loc[:, (df_comb_date != 0).any(axis=0)]

df_comb_concat.index = pd.to_datetime(df_comb_concat.index)

df_comb_concat = df_comb_concat.iloc[1:]

with output_four:
    display(df_comb_concat) 
    
def dropdown_stations_eventhandler(change):
    common_filtering1(change.new)

ValueError: Length mismatch: Expected axis has 18 elements, new values have 41 elements

In [103]:
display(output_four)

Output()